# Итоговая работа по курсу BIG DATA (PySpark)

In [2]:
!apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:15 http://security.ubun

In [3]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [4]:
!wget -q https://downloads.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

In [ ]:
!tar -xvf spark-3.1.2-bin-hadoop2.7.tgz

In [6]:
!pip install -q findspark

In [7]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [8]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

### 1. Загрузка данных

In [9]:
ratings = spark.read.csv('u.data.csv', sep='\t', inferSchema=True, header=None)
ratings.take(5)

[Row(_c0=196, _c1=242, _c2=3, _c3=881250949),
 Row(_c0=186, _c1=302, _c2=3, _c3=891717742),
 Row(_c0=22, _c1=377, _c2=1, _c3=878887116),
 Row(_c0=244, _c1=51, _c2=2, _c3=880606923),
 Row(_c0=166, _c1=346, _c2=1, _c3=886397596)]

In [10]:
ratings = ratings.withColumnRenamed('_c0', 'User_id').withColumnRenamed('_c1', 'Movie_id').withColumnRenamed('_c2', 'Rating').withColumnRenamed('_c3', 'Timestamp')
ratings.show()

+-------+--------+------+---------+
|User_id|Movie_id|Rating|Timestamp|
+-------+--------+------+---------+
|    196|     242|     3|881250949|
|    186|     302|     3|891717742|
|     22|     377|     1|878887116|
|    244|      51|     2|880606923|
|    166|     346|     1|886397596|
|    298|     474|     4|884182806|
|    115|     265|     2|881171488|
|    253|     465|     5|891628467|
|    305|     451|     3|886324817|
|      6|      86|     3|883603013|
|     62|     257|     2|879372434|
|    286|    1014|     5|879781125|
|    200|     222|     5|876042340|
|    210|      40|     3|891035994|
|    224|      29|     3|888104457|
|    303|     785|     3|879485318|
|    122|     387|     5|879270459|
|    194|     274|     2|879539794|
|    291|    1042|     4|874834944|
|    234|    1184|     2|892079237|
+-------+--------+------+---------+
only showing top 20 rows



In [11]:
ratings.describe().show()

+-------+------------------+------------------+------------------+-----------------+
|summary|           User_id|          Movie_id|            Rating|        Timestamp|
+-------+------------------+------------------+------------------+-----------------+
|  count|            100000|            100000|            100000|           100000|
|   mean|         462.48475|         425.53013|           3.52986|8.8352885148862E8|
| stddev|266.61442012750905|330.79835632558473|1.1256735991443214|5343856.189502848|
|    min|                 1|                 1|                 1|        874724710|
|    max|               943|              1682|                 5|        893286638|
+-------+------------------+------------------+------------------+-----------------+



In [12]:
movies = spark.read.csv('u.item.csv', sep='|', inferSchema=True, header=None)
movies.take(5)

[Row(_c0=1, _c1='Toy Story (1995)', _c2='01-Jan-1995', _c3=None, _c4='http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)', _c5=0, _c6=0, _c7=0, _c8=1, _c9=1, _c10=1, _c11=0, _c12=0, _c13=0, _c14=0, _c15=0, _c16=0, _c17=0, _c18=0, _c19=0, _c20=0, _c21=0, _c22=0, _c23=0),
 Row(_c0=2, _c1='GoldenEye (1995)', _c2='01-Jan-1995', _c3=None, _c4='http://us.imdb.com/M/title-exact?GoldenEye%20(1995)', _c5=0, _c6=1, _c7=1, _c8=0, _c9=0, _c10=0, _c11=0, _c12=0, _c13=0, _c14=0, _c15=0, _c16=0, _c17=0, _c18=0, _c19=0, _c20=0, _c21=1, _c22=0, _c23=0),
 Row(_c0=3, _c1='Four Rooms (1995)', _c2='01-Jan-1995', _c3=None, _c4='http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)', _c5=0, _c6=0, _c7=0, _c8=0, _c9=0, _c10=0, _c11=0, _c12=0, _c13=0, _c14=0, _c15=0, _c16=0, _c17=0, _c18=0, _c19=0, _c20=0, _c21=1, _c22=0, _c23=0),
 Row(_c0=4, _c1='Get Shorty (1995)', _c2='01-Jan-1995', _c3=None, _c4='http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995)', _c5=0, _c6=1, _c7=0, _c8=0, _c9=0, _c10=1, _c11=0,

In [13]:
from pyspark.sql.functions import col

In [14]:
movies = movies.select(col('_c0').alias('Movie_id'), col('_c1').alias('Movie_name'), col('_c2').alias('Release_date'), col('_c3').alias('Video_date'), col('_c4').alias('IMDb_URL'), col('_c5').alias('?'),\
                       col('_c6').alias('Action'), col('_c7').alias('Adventure'), col('_c8').alias('Animation'), col('_c9').alias('Child'), col('_c10').alias('Comedy'), col('_c11').alias('Detective'),\
                       col('_c12').alias('Docs'), col('_c13').alias('Drama'), col('_c14').alias('Fantasy'), col('_c15').alias('Noir'), col('_c16').alias('Horror'), col('_c17').alias('Musical'),\
                       col('_c18').alias('Mystic'), col('_c19').alias('Romantic'), col('_c20').alias('Science'), col('_c21').alias('Thriller'), col('_c22').alias('War'), col('_c23').alias('Western'))

In [15]:
movies.show()

+--------+--------------------+------------+----------+--------------------+---+------+---------+---------+-----+------+---------+----+-----+-------+----+------+-------+------+--------+-------+--------+---+-------+
|Movie_id|          Movie_name|Release_date|Video_date|            IMDb_URL|  ?|Action|Adventure|Animation|Child|Comedy|Detective|Docs|Drama|Fantasy|Noir|Horror|Musical|Mystic|Romantic|Science|Thriller|War|Western|
+--------+--------------------+------------+----------+--------------------+---+------+---------+---------+-----+------+---------+----+-----+-------+----+------+-------+------+--------+-------+--------+---+-------+
|       1|    Toy Story (1995)| 01-Jan-1995|      null|http://us.imdb.co...|  0|     0|        0|        1|    1|     1|        0|   0|    0|      0|   0|     0|      0|     0|       0|      0|       0|  0|      0|
|       2|    GoldenEye (1995)| 01-Jan-1995|      null|http://us.imdb.co...|  0|     1|        1|        0|    0|     0|        0|   0|    0

In [16]:
movies.describe().show()

+-------+------------------+--------------------+------------+----------+--------------------+--------------------+-------------------+------------------+-------------------+-------------------+-------------------+-------------------+--------------------+-------------------+--------------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+-------------------+
|summary|          Movie_id|          Movie_name|Release_date|Video_date|            IMDb_URL|                   ?|             Action|         Adventure|          Animation|              Child|             Comedy|          Detective|                Docs|              Drama|             Fantasy|                Noir|              Horror|            Musical|             Mystic|           Romantic|            Science|          Thriller|                War|            Western|
+-------+------------------+----------------

In [17]:
joined = ratings.join(movies, 'Movie_id', how='left')
joined.show()

+--------+-------+------+---------+--------------------+------------+----------+--------------------+---+------+---------+---------+-----+------+---------+----+-----+-------+----+------+-------+------+--------+-------+--------+---+-------+
|Movie_id|User_id|Rating|Timestamp|          Movie_name|Release_date|Video_date|            IMDb_URL|  ?|Action|Adventure|Animation|Child|Comedy|Detective|Docs|Drama|Fantasy|Noir|Horror|Musical|Mystic|Romantic|Science|Thriller|War|Western|
+--------+-------+------+---------+--------------------+------------+----------+--------------------+---+------+---------+---------+-----+------+---------+----+-----+-------+----+------+-------+------+--------+-------+--------+---+-------+
|     242|    196|     3|881250949|        Kolya (1996)| 24-Jan-1997|      null|http://us.imdb.co...|  0|     0|        0|        0|    0|     1|        0|   0|    0|      0|   0|     0|      0|     0|       0|      0|       0|  0|      0|
|     302|    186|     3|891717742|L.A. 

In [18]:
joined.describe().show()

+-------+------------------+------------------+------------------+-----------------+--------------------+------------+----------+--------------------+--------------------+-------------------+-------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-----------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-----------------+-------------------+
|summary|          Movie_id|           User_id|            Rating|        Timestamp|          Movie_name|Release_date|Video_date|            IMDb_URL|                   ?|             Action|          Adventure|         Animation|             Child|             Comedy|          Detective|               Docs|              Drama|          Fantasy|               Noir|             Horror|            Musical|             Mystic|           Romantic|            Science|       

### 2. Средняя оценка для каждого фильма

In [19]:
avg_movies_rating = joined.groupBy('Movie_name').avg('Rating').orderBy('avg(Rating)', ascending=False)
avg_movies_rating.show()

+--------------------+-----------------+
|          Movie_name|      avg(Rating)|
+--------------------+-----------------+
|     Star Kid (1997)|              5.0|
|Aiqing wansui (1994)|              5.0|
|They Made Me a Cr...|              5.0|
|Entertaining Ange...|              5.0|
|Marlene Dietrich:...|              5.0|
|  Prefontaine (1997)|              5.0|
|Santa with Muscle...|              5.0|
|Someone Else's Am...|              5.0|
|Saint of Fort Was...|              5.0|
|Great Day in Harl...|              5.0|
|Pather Panchali (...|            4.625|
|      Everest (1998)|              4.5|
|Some Mother's Son...|              4.5|
|Maya Lin: A Stron...|              4.5|
|         Anna (1996)|              4.5|
|Close Shave, A (1...|4.491071428571429|
|Schindler's List ...|4.466442953020135|
|Wrong Trousers, T...|4.466101694915254|
|   Casablanca (1942)| 4.45679012345679|
|Wallace & Gromit:...|4.447761194029851|
+--------------------+-----------------+
only showing top

### 3. Средняя оценка для каждого жанра

In [20]:
genres = joined.select('Video_date', 'Movie_name', 'Rating', 'Action', 'Adventure', 'Animation', 'Child', 'Comedy', 'Detective', 'Docs', 'Drama', 'Fantasy',\
                       'Noir', 'Horror', 'Musical', 'Mystic', 'Romantic', 'Science', 'Thriller', 'War', 'Western')
genres = genres.na.fill({'Video_date': 1})
genres.show()

+----------+--------------------+------+------+---------+---------+-----+------+---------+----+-----+-------+----+------+-------+------+--------+-------+--------+---+-------+
|Video_date|          Movie_name|Rating|Action|Adventure|Animation|Child|Comedy|Detective|Docs|Drama|Fantasy|Noir|Horror|Musical|Mystic|Romantic|Science|Thriller|War|Western|
+----------+--------------------+------+------+---------+---------+-----+------+---------+----+-----+-------+----+------+-------+------+--------+-------+--------+---+-------+
|         1|        Kolya (1996)|     3|     0|        0|        0|    0|     1|        0|   0|    0|      0|   0|     0|      0|     0|       0|      0|       0|  0|      0|
|         1|L.A. Confidential...|     3|     0|        0|        0|    0|     0|        1|   0|    0|      0|   1|     0|      0|     1|       0|      0|       1|  0|      0|
|         1| Heavyweights (1994)|     1|     0|        0|        0|    1|     1|        0|   0|    0|      0|   0|     0|    

In [21]:
genres.count()

100000

In [22]:
genres_ratings = genres.withColumn('Action_score', genres.Rating * genres.Action).withColumn('Adventure_score', genres.Rating * genres.Adventure).withColumn('Animation_score', genres.Rating * genres.Animation)\
                       .withColumn('Child_score', genres.Rating * genres.Child).withColumn('Comedy_score', genres.Rating * genres.Comedy).withColumn('Detective_score', genres.Rating * genres.Detective)\
                       .withColumn('Docs_score', genres.Rating * genres.Docs).withColumn('Drama_score', genres.Rating * genres.Drama).withColumn('Fantasy_score', genres.Rating * genres.Fantasy)\
                       .withColumn('Noir_score', genres.Rating * genres.Noir).withColumn('Horror_score', genres.Rating * genres.Horror).withColumn('Musical_score', genres.Rating * genres.Musical)\
                       .withColumn('Mystic_score', genres.Rating * genres.Mystic).withColumn('Romantic_score', genres.Rating * genres.Romantic).withColumn('Science_score', genres.Rating * genres.Science)\
                       .withColumn('Thriller_score', genres.Rating * genres.Thriller).withColumn('War_score', genres.Rating * genres.War).withColumn('Western_score', genres.Rating * genres.Western)
genres_ratings.show()

+----------+--------------------+------+------+---------+---------+-----+------+---------+----+-----+-------+----+------+-------+------+--------+-------+--------+---+-------+------------+---------------+---------------+-----------+------------+---------------+----------+-----------+-------------+----------+------------+-------------+------------+--------------+-------------+--------------+---------+-------------+
|Video_date|          Movie_name|Rating|Action|Adventure|Animation|Child|Comedy|Detective|Docs|Drama|Fantasy|Noir|Horror|Musical|Mystic|Romantic|Science|Thriller|War|Western|Action_score|Adventure_score|Animation_score|Child_score|Comedy_score|Detective_score|Docs_score|Drama_score|Fantasy_score|Noir_score|Horror_score|Musical_score|Mystic_score|Romantic_score|Science_score|Thriller_score|War_score|Western_score|
+----------+--------------------+------+------+---------+---------+-----+------+---------+----+-----+-------+----+------+-------+------+--------+-------+--------+---+

In [23]:
avg_genres_rating = genres_ratings.where(genres_ratings.Action_score != 0).groupBy('Video_date').avg('Action_score')\
                                    .join((genres_ratings.where(genres_ratings.Adventure_score != 0).groupBy('Video_date').avg('Adventure_score')), 'Video_date', how='left')\
                                    .join((genres_ratings.where(genres_ratings.Animation_score != 0).groupBy('Video_date').avg('Animation_score')), 'Video_date', how='left')\
                                    .join((genres_ratings.where(genres_ratings.Child_score != 0).groupBy('Video_date').avg('Child_score')), 'Video_date', how='left')\
                                    .join((genres_ratings.where(genres_ratings.Comedy_score != 0).groupBy('Video_date').avg('Comedy_score')), 'Video_date', how='left')\
                                    .join((genres_ratings.where(genres_ratings.Detective_score != 0).groupBy('Video_date').avg('Detective_score')), 'Video_date', how='left')\
                                    .join((genres_ratings.where(genres_ratings.Docs_score != 0).groupBy('Video_date').avg('Docs_score')), 'Video_date', how='left')\
                                    .join((genres_ratings.where(genres_ratings.Drama_score != 0).groupBy('Video_date').avg('Drama_score')), 'Video_date', how='left')\
                                    .join((genres_ratings.where(genres_ratings.Fantasy_score != 0).groupBy('Video_date').avg('Fantasy_score')), 'Video_date', how='left')\
                                    .join((genres_ratings.where(genres_ratings.Noir_score != 0).groupBy('Video_date').avg('Noir_score')), 'Video_date', how='left')\
                                    .join((genres_ratings.where(genres_ratings.Horror_score != 0).groupBy('Video_date').avg('Horror_score')), 'Video_date', how='left')\
                                    .join((genres_ratings.where(genres_ratings.Musical_score != 0).groupBy('Video_date').avg('Musical_score')), 'Video_date', how='left')\
                                    .join((genres_ratings.where(genres_ratings.Mystic_score != 0).groupBy('Video_date').avg('Mystic_score')), 'Video_date', how='left')\
                                    .join((genres_ratings.where(genres_ratings.Romantic_score != 0).groupBy('Video_date').avg('Romantic_score')), 'Video_date', how='left')\
                                    .join((genres_ratings.where(genres_ratings.Science_score != 0).groupBy('Video_date').avg('Science_score')), 'Video_date', how='left')\
                                    .join((genres_ratings.where(genres_ratings.Thriller_score != 0).groupBy('Video_date').avg('Thriller_score')), 'Video_date', how='left')\
                                    .join((genres_ratings.where(genres_ratings.War_score != 0).groupBy('Video_date').avg('War_score')), 'Video_date', how='left')\
                                    .join((genres_ratings.where(genres_ratings.Western_score != 0).groupBy('Video_date').avg('Western_score')), 'Video_date', how='left')
avg_genres_rating.show()

+----------+-----------------+--------------------+--------------------+------------------+------------------+--------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+-------------------+------------------+-------------------+-----------------+------------------+
|Video_date|avg(Action_score)|avg(Adventure_score)|avg(Animation_score)|  avg(Child_score)| avg(Comedy_score)|avg(Detective_score)|   avg(Docs_score)|  avg(Drama_score)|avg(Fantasy_score)|   avg(Noir_score)| avg(Horror_score)|avg(Musical_score)|avg(Mystic_score)|avg(Romantic_score)|avg(Science_score)|avg(Thriller_score)|   avg(War_score)|avg(Western_score)|
+----------+-----------------+--------------------+--------------------+------------------+------------------+--------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+-----------------

#### *Средствами SQL:*

In [24]:
genres.createOrReplaceTempView('Genres')

In [25]:
spark.sql('SELECT sum(Action*Rating)/sum(Action) as ActionAvg, sum(Adventure*Rating)/sum(Adventure) as AdventureAvg, sum(Animation*Rating)/sum(Animation) as AnimationAvg,\
                  sum(Child*Rating)/sum(Child) as ChildAvg, sum(Comedy*Rating)/sum(Comedy) as ComedyAvg, sum(Detective*Rating)/sum(Detective) as DetectiveAvg,\
                  sum(Docs*Rating)/sum(Docs) as DocsAvg, sum(Drama*Rating)/sum(Drama) as DramaAvg, sum(Fantasy*Rating)/sum(Fantasy) as FantasyAvg,\
                  sum(Noir*Rating)/sum(Noir) as NoirAvg, sum(Horror*Rating)/sum(Horror) as HorrorAvg, sum(Musical*Rating)/sum(Musical) as MusicalAvg,\
                  sum(Mystic*Rating)/sum(Mystic) as MysticAvg, sum(Romantic*Rating)/sum(Romantic) as RomanticAvg, sum(Science*Rating)/sum(Science) as ScienceAvg,\
                  sum(Thriller*Rating)/sum(Thriller) as ThrillerAvg, sum(War*Rating)/sum(War) as WarAvg, sum(Western*Rating)/sum(Western) as WesternAvg\
           FROM Genres').show()

+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+----------------+-----------------+------------------+------------------+-----------------+------------------+
|        ActionAvg|     AdventureAvg|      AnimationAvg|          ChildAvg|         ComedyAvg|      DetectiveAvg|           DocsAvg|          DramaAvg|        FantasyAvg|           NoirAvg|         HorrorAvg|       MusicalAvg|       MysticAvg|      RomanticAvg|        ScienceAvg|       ThrillerAvg|           WarAvg|        WesternAvg|
+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+----------------+-----------------+------------------+------------------+-----------------+---------

#### или так

In [50]:
spark.sql('SELECT avg(Action*Rating) as ActionAvg FROM Genres where Action != 0').join(spark.sql('SELECT avg(Adventure*Rating) as AdventureAvg FROM Genres where Adventure != 0'))\
     .join(spark.sql('SELECT avg(Animation*Rating) as AnimationAvg FROM Genres where Animation != 0')).join(spark.sql('SELECT avg(Child*Rating) as ChildAvg FROM Genres where Child != 0'))\
     .join(spark.sql('SELECT avg(Comedy*Rating) as ComedyAvg FROM Genres where Comedy != 0')).join(spark.sql('SELECT avg(Detective*Rating) as DetectiveAvg FROM Genres where Detective != 0'))\
     .join(spark.sql('SELECT avg(Docs*Rating) as DocsAvg FROM Genres where Docs != 0')).join(spark.sql('SELECT avg(Drama*Rating) as DramaAvg FROM Genres where Drama != 0'))\
     .join(spark.sql('SELECT avg(Fantasy*Rating) as FantasyAvg FROM Genres where Fantasy != 0')).join(spark.sql('SELECT avg(Noir*Rating) as NoirAvg FROM Genres where Noir != 0'))\
     .join(spark.sql('SELECT avg(Horror*Rating) as HorrorAvg FROM Genres where Horror != 0')).join(spark.sql('SELECT avg(Musical*Rating) as MusicalAvg FROM Genres where Musical != 0'))\
     .join(spark.sql('SELECT avg(Mystic*Rating) as MysticAvg FROM Genres where Mystic != 0')).join(spark.sql('SELECT avg(Romantic*Rating) as RomanticAvg FROM Genres where Romantic != 0'))\
     .join(spark.sql('SELECT avg(Science*Rating) as ScienceAvg FROM Genres where Science != 0')).join(spark.sql('SELECT avg(Thriller*Rating) as ThrillerAvg FROM Genres where Thriller != 0'))\
     .join(spark.sql('SELECT avg(War*Rating) as WarAvg FROM Genres where War != 0')).join(spark.sql('SELECT avg(Western*Rating) as WesternAvg FROM Genres where Western != 0')).show()     

+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+----------------+-----------------+------------------+------------------+-----------------+------------------+
|        ActionAvg|     AdventureAvg|      AnimationAvg|          ChildAvg|         ComedyAvg|      DetectiveAvg|           DocsAvg|          DramaAvg|        FantasyAvg|           NoirAvg|         HorrorAvg|       MusicalAvg|       MysticAvg|      RomanticAvg|        ScienceAvg|       ThrillerAvg|           WarAvg|        WesternAvg|
+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+----------------+-----------------+------------------+------------------+-----------------+---------

### 4.1 Получение датафрейма с 5-ю самыми популярными фильмами

In [29]:
import pandas as pd

In [30]:
movies_ratings = joined.select('Movie_name', 'Rating')
movies_ratings.show()

+--------------------+------+
|          Movie_name|Rating|
+--------------------+------+
|        Kolya (1996)|     3|
|L.A. Confidential...|     3|
| Heavyweights (1994)|     1|
|Legends of the Fa...|     2|
| Jackie Brown (1997)|     1|
|Dr. Strangelove o...|     4|
|Hunt for Red Octo...|     2|
|Jungle Book, The ...|     5|
|       Grease (1978)|     3|
|Remains of the Da...|     3|
| Men in Black (1997)|     2|
|Romy and Michele'...|     5|
|Star Trek: First ...|     5|
|To Wong Foo, Than...|     3|
|Batman Forever (1...|     3|
|     Only You (1994)|     3|
|Age of Innocence,...|     5|
|      Sabrina (1995)|     2|
|   Just Cause (1995)|     4|
|Endless Summer 2,...|     2|
+--------------------+------+
only showing top 20 rows



In [31]:
movies_ratings.count()

100000

In [32]:
rating_5 = movies_ratings.where(movies_ratings.Rating == 5)
five_top5 = rating_5.groupBy('Movie_name').count().orderBy('count', ascending=False)
df_five_top5 = five_top5.toPandas()
df_five_top5.head(5)

,Movie_name,count
0,Star Wars (1977),325
1,Fargo (1996),227
2,"Godfather, The (1972)",214
3,Raiders of the Lost Ark (1981),202
4,Pulp Fiction (1994),188


### 4.2 Получение датафрейма с 5-ю самыми непопулярными фильмами

In [33]:
rating_1 = movies_ratings.where(movies_ratings.Rating == 1)
five_not_pop = rating_1.groupBy('Movie_name').count().orderBy('count')
df_five_not_pop = five_not_pop.toPandas()
df_five_not_pop.head(5)

,Movie_name,count
0,Psycho (1960),1
1,Three Wishes (1995),1
2,"Paris, France (1993)",1
3,Snow White and the Seven Dwarfs (1937),1
4,Mondo (1996),1
